<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/stock/momentum_shortlist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas_ta --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import pandas_ta as ta
from scipy.stats import linregress
import warnings
warnings.filterwarnings('ignore')

# Define functions for each evaluation criterion
def calculate_moving_averages(data):
    data['50_MA'] = data['Close'].rolling(window=50).mean()
    data['200_MA'] = data['Close'].rolling(window=200).mean()
    return data

def calculate_rsi(data):
    data['RSI'] = ta.rsi(data['Close'], length=14)
    return data

def calculate_macd(data):
    macd = ta.macd(data['Close'])
    data['MACD'] = macd['MACD_12_26_9']
    data['MACD_Signal'] = macd['MACDs_12_26_9']
    return data

def calculate_bollinger_bands(data):
    data['BB_upper'] = data['Close'].rolling(window=20).mean() + 2 * data['Close'].rolling(window=20).std()
    data['BB_lower'] = data['Close'].rolling(window=20).mean() - 2 * data['Close'].rolling(window=20).std()
    return data

def check_uptrend(data, window=20):
    indices = np.arange(window)
    closes = data['Close'][-window:]
    slope, _, _, _, _ = linregress(indices, closes)
    return slope > 0

def evaluate_stock(ticker, data, nifty):
    stock_data = data[ticker]
    if len(stock_data) < 200:
        print(f"Not enough data for {ticker}. Skipping...")
        return None

    stock_data = calculate_moving_averages(stock_data)
    stock_data = calculate_rsi(stock_data)
    stock_data = calculate_macd(stock_data)
    stock_data = calculate_bollinger_bands(stock_data)

    # Ensure indices align
    stock_data = stock_data.dropna()
    if len(stock_data) == 0:
        print(f"No valid data after calculations for {ticker}.")
        return None

    # Calculate returns
    periods = [5, 21, 63]  # Weekly, Monthly, Quarterly
    returns = {}
    for period in periods:
        if len(stock_data) >= period:
            returns[period] = (stock_data['Close'][-1] - stock_data['Close'][-period]) / stock_data['Close'][-period] * 100
        else:
            returns[period] = np.nan

    # Benchmark return
    benchmark_return = (nifty['Close'][-1] - nifty['Close'][-63]) / nifty['Close'][-63] * 100 if len(nifty) >= 63 else np.nan

    # Initialize criteria
    criteria = {
        'Ticker': ticker,
        'Stock Return': returns.get(63, np.nan),
        'Benchmark Return': benchmark_return,
        'Outperformance': returns.get(63, 0) > benchmark_return,
        'Weekly Price Increase': returns.get(5, 0) > 0,
        'Monthly Price Increase': returns.get(21, 0) > 0,
        'Quarterly Price Increase': returns.get(63, 0) > 0,
        '50_MA above 200_MA (Golden Cross)': stock_data['50_MA'][-1] > stock_data['200_MA'][-1],
        'Price above 50_MA': stock_data['Close'][-1] > stock_data['50_MA'][-1],
        'Price above 200_MA': stock_data['Close'][-1] > stock_data['200_MA'][-1],
        'RSI between 50 and 70': 50 < stock_data['RSI'][-1] < 70,
        'Avg Volume Week > Month': stock_data['Volume'][-5:].mean() > stock_data['Volume'][-21:].mean(),
        'MACD above Signal Line': stock_data['MACD'][-1] > stock_data['MACD_Signal'][-1],
        'Price above Upper Bollinger Band': stock_data['Close'][-1] > stock_data['BB_upper'][-1],
        'Uptrend': check_uptrend(stock_data)
    }

    # Define weights
    weights = {
        'Outperformance': 3,
        'Weekly Price Increase': 1,
        'Monthly Price Increase': 2,
        'Quarterly Price Increase': 2,
        '50_MA above 200_MA (Golden Cross)': 1,
        'Price above 50_MA': 1,
        'Price above 200_MA': 1,
        'RSI between 50 and 70': 0.5,
        'Avg Volume Week > Month': 1,
        'MACD above Signal Line': 1,
        'Price above Upper Bollinger Band': 0.5,
        'Uptrend': 3
    }

    # Calculate total score
    total_score = sum(weights[key] * int(criteria[key]) for key in weights)
    criteria['Total Score'] = total_score

    return criteria

# Main code
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=365*2)  # 1 year of data

# Download benchmark data
nifty_index_symbol = "^CNX100"  # Verify the correct symbol
nifty = yf.download(nifty_index_symbol, start=start_date, end=end_date)

# List of stock tickers to evaluate
nifty100_url = "https://raw.githubusercontent.com/anirbanghoshsbi/data/main/ind_nifty50list.csv"
nifty100 = pd.read_csv(nifty100_url)
stock_list =nifty100['Symbol'].apply(lambda x: x + ".NS").tolist()

# Download all stock data at once
stock_data = yf.download(stock_list, start=start_date, end=end_date, group_by='ticker')

# Evaluate stocks
results = []
for stock in stock_list:
    criteria = evaluate_stock(stock, stock_data, nifty)
    if criteria:
        results.append(criteria)

# Convert the results into a pandas DataFrame
df_results = pd.DataFrame(results)

# Set the Ticker as the index
df_results.set_index('Ticker', inplace=True)

# Sort the DataFrame based on Total Score
df_results.sort_values('Total Score', ascending=False, inplace=True)

# Save results to CSV
df_results.to_csv('stock_evaluation_results.csv')

# Display the top 10 stocks
print(df_results.head(10))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  50 of 50 completed


               Stock Return  Benchmark Return  Outperformance  \
Ticker                                                          
DIVISLAB.NS       35.312719          1.864148            True   
ADANIENT.NS        2.340603          1.864148            True   
M&M.NS            15.917460          1.864148            True   
HCLTECH.NS        17.911170          1.864148            True   
HDFCLIFE.NS       13.893476          1.864148            True   
HINDALCO.NS        7.990750          1.864148            True   
NTPC.NS           12.024395          1.864148            True   
BHARTIARTL.NS     17.732428          1.864148            True   
INFY.NS           13.048049          1.864148            True   
SUNPHARMA.NS      20.582634          1.864148            True   

               Weekly Price Increase  Monthly Price Increase  \
Ticker                                                         
DIVISLAB.NS                     True                    True   
ADANIENT.NS                

In [ ]:
#Replace True with 1 and False with 0
df_numeric = df_results.replace({True: 1, False: 0})

# Sum across the rows, excluding 'Stock Return' and 'Benchmark Return'
df_numeric['Row_Sum'] = df_numeric.drop(columns=['Stock Return', 'Benchmark Return']).sum(axis=1)


In [ ]:
# Display the updated DataFrame
df_numeric.sort_values(by='Row_Sum', ascending=False)[['Row_Sum','Stock Return','Benchmark Return']].head(10)

,Row_Sum,Stock Return,Benchmark Return
Ticker,,,
DIVISLAB.NS,27.5,35.312719,1.864148
HCLTECH.NS,25.5,17.911170,1.864148
HDFCLIFE.NS,25.5,13.893476,1.864148
ADANIENT.NS,25.5,2.340603,1.864148
M&M.NS,25.5,15.917460,1.864148
NTPC.NS,23.5,12.024395,1.864148
BHARTIARTL.NS,23.5,17.732428,1.864148
HINDALCO.NS,23.5,7.990750,1.864148
INFY.NS,22.5,13.048049,1.864148


In [ ]:
df_numeric.sort_values(by='Row_Sum', ascending=False)[['Row_Sum','Stock Return','Benchmark Return']].tail(10)

,Row_Sum,Stock Return,Benchmark Return
Ticker,,,
ULTRACEMCO.NS,6.0,-1.560036,1.864148
RELIANCE.NS,4.0,-14.067858,1.864148
TCS.NS,4.0,-0.830555,1.864148
TATACONSUM.NS,4.0,-3.397702,1.864148
TATAMOTORS.NS,4.0,-8.463240,1.864148
NESTLEIND.NS,4.0,-3.725563,1.864148
LT.NS,4.0,-4.570678,1.864148
ICICIBANK.NS,4.0,-0.802987,1.864148
HEROMOTOCO.NS,4.0,-1.272785,1.864148
